In [2]:
import numpy as np

%matplotlib qt
import matplotlib.pyplot as plt

from scipy import ndimage as ndi
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import


from skimage.morphology import watershed
from skimage.feature import peak_local_max
from PIL import Image

from skimage.morphology import skeletonize, thin, skeletonize_3d
from skimage.util import invert
from skimage import measure
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image
from skimage.morphology import disk
from skimage import feature
from skimage.filters import roberts, sobel, scharr, prewitt
import seaborn as sns
sns.reset_orig()

from skimage import restoration

from skimage import exposure
from skimage.filters import gaussian

from scipy import ndimage

from time import time

from skimage.color import rgb2gray


from skimage.filters.rank import entropy
import scipy



import networkx as nx
from scipy import interpolate

from skimage.segmentation import random_walker
from skimage.data import binary_blobs
from skimage.exposure import rescale_intensity
import glob

from skimage.morphology import disk
from skimage import io
from skimage.morphology import remove_small_objects

import threading

import csv

/Users/spencerlab/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


In [4]:
def getNeighbors(arr):
    
    arr2 = (arr.copy())*0
    
    
    #IN YZ PLANE
    
    #bottom front
    arr2[:-1,:-1] += arr[1:,1:]
    #bottom 
    arr2[:-1] += arr[1:]
    #bottom back
    arr2[:-1, 1:] += arr[1:, :-1]
    
    #top front
    arr2[1:, :-1] += arr[:-1, 1:]
    #top
    arr2[1:] += arr[:-1]
    #top back
    arr2[1:, 1:] += arr[:-1, :-1]
    
    #front
    arr2[:, :-1] += arr[:, 1:]
    #back
    arr2[:, 1:] += arr[:, :-1]
    
    
    
    #LEFT LEFT LEFT LEFT LEFT LEFT LEFT 
    #LEFT LEFT LEFT LEFT LEFT LEFT LEFT
    #LEFT LEFT LEFT LEFT LEFT LEFT LEFT
    
    #bottom front
    arr2[:-1,:-1, 1:] += arr[1:,1:, :-1]
    #bottom 
    arr2[:-1, :, 1:] += arr[1:, :, :-1]
    #bottom back
    arr2[:-1, 1:, 1:] += arr[1:, :-1, :-1]
    
    #top front
    arr2[1:, :-1, 1:] += arr[:-1, 1:, :-1]
    #top
    arr2[1:, :, 1:] += arr[:-1, :, :-1]
    #top back
    arr2[1:, 1:, 1:] += arr[:-1, :-1, :-1]
    
    #front
    arr2[:, :-1, 1:] += arr[:, 1:, :-1]
    #back
    arr2[:, 1:, 1:] += arr[:, :-1, :-1]
    
    #just left 
    arr2[:, :, 1:] += arr[:, :, :-1]
    
    
    
    #RIGHT RIGHT RIGHT RIGHT RIGHT RIGHT
    #RIGHT RIGHT RIGHT RIGHT RIGHT RIGHT
    #RIGHT RIGHT RIGHT RIGHT RIGHT RIGHT
    
    #bottom front
    arr2[:-1,:-1, :-1] += arr[1:,1:, 1:]
    #bottom 
    arr2[:-1, :, :-1] += arr[1:, :, 1:]
    #bottom back
    arr2[:-1, 1:, :-1] += arr[1:, :-1, 1:]
    
    #top front
    arr2[1:, :-1, :-1] += arr[:-1, 1:, 1:]
    #top
    arr2[1:, :, :-1] += arr[:-1, :, 1:]
    #top back
    arr2[1:, 1:, :-1] += arr[:-1, :-1, 1:]
    
    #front
    arr2[:, :-1, :-1] += arr[:, 1:, 1:]
    #back
    arr2[:, 1:, :-1] += arr[:, :-1, 1:]
    
    
    #just right 
    arr2[:, :, :-1] += arr[:, :, 1:]
    
    
    
    return arr2*(arr!=0)

In [4]:
segs = io.imread("/Users/spencerlab/Desktop/Temporary_Computational_Folders/Blood_Vessels/Graph_development/skel_Segs_3D.tif")

In [7]:
# nodes_csv = pd.read_csv("/Users/Spenc/OneDrive/Desktop/Kumaran/bv/node_info.csv") # Tomar's CSV (in downloads too)
# segs_csv = pd.read_csv("/Users/Spenc/OneDrive/Desktop/Kumaran/bv/segs_info.csv") # Shreya's CSV seg_nodes_2 (downloads)    

In [11]:
# nodes_info = nodes_csv.to_numpy()
# segs_info = segs_csv.to_numpy()

In [77]:
node_num = 23 # enter node num

segs_at_node = nodes_info[node_num]
segs_at_node = np.delete(segs_at_node,0)
segs_at_node = segs_at_node[np.logical_not(np.isnan(segs_at_node))]
segs_at_node = segs_at_node.astype(int)

In [78]:
segs_at_node

array([24, 42, 44])

In [79]:
z = []
y = []
x = []

for each in segs_at_node:
    z.append(segs_info[each][1])
    z.append(segs_info[each][5])
    
    y.append(segs_info[each][2])
    y.append(segs_info[each][6])
    
    x.append(segs_info[each][3])
    x.append(segs_info[each][7])
    

In [48]:
z_max = max(z)
y_max = max(y)
x_max = max(x)

z_min = min(z)
y_min = min(y)
x_min = min(x)

In [1]:
segs_substack = segs[z_min:z_max+1, y_min:y_max+1, x_min:x_max+1]

NameError: name 'segs' is not defined

In [112]:
segs_substack

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)

In [105]:
# labeled_substack, num_segs = ndi.measurements.label(segs_substack, np.ones((3,3,3)))

In [110]:
# act_labels = []
# for each in segs_at_node:
#     act_labels.append(labeled_substack[segs_info[each][1]][segs_info[each][2]][segs_info[each][3]])
                                       
                                    
    
    


In [109]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.po(segs_substack,  edgecolor='k')

{}

In [ ]:
#  size of substack (each side)
Z_margin = 100
Y_margin = 100
X_margin = 100

#  center of substack
Center_Z = 44
Center_Y = 1404
Center_X = 2537

sub_stack = segs[Center_Z-Z_margin:Center_Z+Z_margin+1,Center_Y-Y_margin:Center_Y+Y_margin+1, Center_X-X_margin:Center_X+X_margin+1]